# Лабораторная работа 2

## Задание 1
Реализовать в среде MATLAB метод наискорейшего спуска, сопряженных градиентов, Ньютона, правильного симплекса, циклического покоординатного спуска, Хука-Дживса и случайного поиска, при реализации методов использовать аналитические значения производных и их разностные аппроксимации.

In [2]:
import math

import numpy as np
from sympy import lambdify
from sympy.abc import x, y

from functools import total_ordering


@total_ordering
class Point:
    """Класс точки функции и ее значения для более удобных манипуляций"""

    def __init__(self, x_p, value=float('inf')):
        self.x_point = x_p
        self.value = value

    def __eq__(self, other):
        return self.value == other.value

    def __le__(self, other):
            return self.value <= other.value
    
    def get_coords(self):
         return self.x_point


@total_ordering
class Point2:
    """Класс точки функции и ее значения для более удобных манипуляций"""

    def __init__(self, x_p, y_p, value=float('inf')):
        self.x_point = x_p
        self.y_point = y_p
        self.value = value

    def __eq__(self, other):
        return self.value == other.value

    def __le__(self, other):
            return self.value <= other.value
    
    def get_coords(self):
         return np.array([self.x_point, self.y_point])


def calcule_gradient(func, symbols, values) -> tuple[np.ndarray, int]:

    direvatives_matrix = [func.diff(arg) for arg in symbols]
    gm_k1 = np.array([lambdify(symbols, derevat)(*values) for derevat in direvatives_matrix])
    direvatives_calculatus = len(direvatives_matrix)

    return gm_k1, direvatives_calculatus


def bitByBitSearch(func, x, eps=10e-3):
    """Реализация метода поразрядного поиска. Возвращает точку минимума, значение минимума функции и количество вычислений"""

    cur_eps = 0.25 if 0.25 > eps else eps

    direction_to_right = True
    compute_number = 0
    
    func_arg = x
    last_point = Point(func_arg, func(func_arg))

    
    while True:

        while True:
            
            func_arg += cur_eps if direction_to_right else -1*cur_eps
            
            func_value = func(func_arg)
            func_point = Point(func_arg, func_value)
            compute_number += 1
            
            if func_point >= last_point:
                break
            else:
                last_point = func_point

        if cur_eps <= eps:
            break
        else:
            last_point = func_point
            direction_to_right = not direction_to_right
            cur_eps /= 4

    return last_point.get_coords(), last_point.value, compute_number


In [3]:
def fastest_downhill(func, x_val, y_val, eps):

    iteration_number = 0
    calcul_number = 0
    point = Point2(x_val, y_val)

    func_lambd = lambdify([x, y], func)
    point.value = func_lambd(x_val, y_val)

    while True:
        iteration_number += 1

        gradient_matrix, difs_count = calcule_gradient(func, [x, y], point.get_coords())
        calcul_number += difs_count
        gradient_norm: float = np.linalg.norm(gradient_matrix)

        if gradient_norm < eps:
            break

        x_k = point.get_coords()
        F_min = lambda alpha: func_lambd(*(x_k - alpha * gradient_matrix))
        alpha_min, new_value, func_count = bitByBitSearch(F_min, 0)
        calcul_number += func_count

        x_new, y_new = x_k - alpha_min * gradient_matrix
        point = Point2(x_new, y_new, new_value)

    return point.get_coords(), point.value, iteration_number, calcul_number

In [16]:
def conjugate_gradients_method(func, x_val, y_val, eps):

    k = 0
    iteration_count = 0
    calcul_number = 0
    point = Point2(x_val, y_val)
    func_lambd = lambdify([x, y], func)

    gm_k1, difs_count = calcule_gradient(func, [x, y], point.get_coords())
    calcul_number += difs_count

    p_0 = -gm_k1

    while True:
        iteration_count += 1

        F_min = lambda alpha: func_lambd(*(point.get_coords() + alpha * p_0))
        alpha_min, _, func_count = bitByBitSearch(F_min, 0)
        calcul_number += func_count

        x_k2 = point.get_coords() + alpha_min * p_0
        gm_k2, difs_count = calcule_gradient(func, [x, y], x_k2)
        calcul_number += difs_count

        gm_k2_norm = np.linalg.norm(gm_k2)
        if gm_k2_norm < eps:
            break

        if k + 1 == 2:
            beta = 0
            k = 0
        else:
            beta = gm_k2_norm**2 / np.linalg.norm(gm_k1)**2
            k += 1

        p_0 = -gm_k2 + beta * p_0
        point = Point2(*x_k2)
    
    return x_k2, func_lambd(*x_k2), iteration_count, calcul_number + 1

In [18]:
research_func = 4*x**2 + 3*y**2 - 4*x*y + x

minimum_p = conjugate_gradients_method(research_func, 0, 0, 0.0001)
print(f"Minimum point --- func({minimum_p[0]}) = {minimum_p[1]}; {minimum_p[2]} iterations and {minimum_p[3]} calculates")

Minimum point --- func([-0.1875 -0.125 ]) = -0.09375; 2 iterations and 38 calculates
